# Run this jupyter notebook to get hourly average prices (LMP, MCC, MCE, MCL) for a given time range

In [1]:
import os
from glob import glob
import pandas as pd
import sys

sys.path.append(os.getcwd())

## Scrape LMP (along with MCL, MCE, MCC) in DAM

In [2]:
# replace the following variables with your own
_market = "DAM"
_start_date = "20230723"
_end_date = "20230727"

data_folder = "data/LMP" #just updated as LMP, instead of prices
zip_data_folder = os.path.join(data_folder, "raw")
%run -i 'scraper_CAISO_LMP.py' --market {_market} --start_date {_start_date} --end_date {_end_date} --save_to_path {zip_data_folder}

Namespace(market='DAM', start_date='20230723', end_date='20230727', save_to_path='data/prices/zip')


Downloading: 20230727_20230727_DAM_LMP_GRP_N_N_v12_csv.zip: 100%|█| 5/5 [01:15<0

Successfully downloaded data from 20230723 to 20230727


## Unzip files

In [3]:
from utils import unzip_files

unzip_data_folder = os.path.join(data_folder, 'unzip')
unzip_files(input_dir=zip_data_folder, output_dir=unzip_data_folder)

Extracting: 20230723_20230723_DAM_LMP_GRP_N_N_v12_csv.zip: 100%|█| 5/5 [00:00<00

Successfully extract files into data/prices/unzip


## Prepare LMP, MCC, MCE, MCL files

In [4]:
# load LMP csv files
current_dir = os.getcwd() # use os.path.dirname(__file__) does not work in jupyter notebook
data_dir = os.path.join(current_dir, unzip_data_folder)

LMP_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_LMP_*.csv')))
MCC_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_MCC_*.csv')))
MCE_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_MCE_*.csv')))
MCL_csv_files = sorted(glob(os.path.join(data_dir, '*_LMP_DAM_MCL_*.csv')))

print("===LMP===\n")
print([os.path.basename(file) for file in LMP_csv_files[:5]], "\n")
print("===MCC===\n")
print([os.path.basename(file) for file in MCC_csv_files[:5]], "\n")
print("===MCE===\n")
print([os.path.basename(file) for file in MCE_csv_files[:5]], "\n")
print("===MCL===\n")
print([os.path.basename(file) for file in MCL_csv_files[:5]], "\n")

===LMP===

['20230723_20230723_PRC_LMP_DAM_LMP_v12.csv', '20230724_20230724_PRC_LMP_DAM_LMP_v12.csv', '20230725_20230725_PRC_LMP_DAM_LMP_v12.csv', '20230726_20230726_PRC_LMP_DAM_LMP_v12.csv', '20230727_20230727_PRC_LMP_DAM_LMP_v12.csv'] 

===MCC===

['20230723_20230723_PRC_LMP_DAM_MCC_v12.csv', '20230724_20230724_PRC_LMP_DAM_MCC_v12.csv', '20230725_20230725_PRC_LMP_DAM_MCC_v12.csv', '20230726_20230726_PRC_LMP_DAM_MCC_v12.csv', '20230727_20230727_PRC_LMP_DAM_MCC_v12.csv'] 

===MCE===

['20230723_20230723_PRC_LMP_DAM_MCE_v12.csv', '20230724_20230724_PRC_LMP_DAM_MCE_v12.csv', '20230725_20230725_PRC_LMP_DAM_MCE_v12.csv', '20230726_20230726_PRC_LMP_DAM_MCE_v12.csv', '20230727_20230727_PRC_LMP_DAM_MCE_v12.csv'] 

===MCL===

['20230723_20230723_PRC_LMP_DAM_MCL_v12.csv', '20230724_20230724_PRC_LMP_DAM_MCL_v12.csv', '20230725_20230725_PRC_LMP_DAM_MCL_v12.csv', '20230726_20230726_PRC_LMP_DAM_MCL_v12.csv', '20230727_20230727_PRC_LMP_DAM_MCL_v12.csv'] 



## Calculate hourly average price for LMP, MCC, MCE, MCL and save the results into separate csv files

In [5]:
from draw_avg_price import find_hourly_avg_price

avg_price_folder = os.path.join(data_folder, 'avg_hourly_price')

find_hourly_avg_price(
    LMP_csv_files=LMP_csv_files,
    MCC_csv_files=MCC_csv_files,
    MCE_csv_files=MCE_csv_files,
    MCL_csv_files=MCL_csv_files,
    output_dir=avg_price_folder,
)

print('Done.')

Done.


In [6]:
sorted(glob(os.path.join(avg_price_folder, '*')))

['data/prices/avg_hourly_price/hourly_average_LMP.csv',
 'data/prices/avg_hourly_price/hourly_average_MCC.csv',
 'data/prices/avg_hourly_price/hourly_average_MCE.csv',
 'data/prices/avg_hourly_price/hourly_average_MCL.csv']

## Peek average prices

In [7]:
for file in sorted(glob(os.path.join(current_dir, avg_price_folder, '*'))):
    df = pd.read_csv(file, low_memory=False)
    print(df.head())

       INTERVALSTARTTIME_GMT         MW      OPR_DT LMP_TYPE
0  2023-07-23T07:00:00-00:00  66.924758  2023-07-23      LMP
1  2023-07-23T08:00:00-00:00  62.210409  2023-07-23      LMP
2  2023-07-23T09:00:00-00:00  56.600809  2023-07-23      LMP
3  2023-07-23T10:00:00-00:00  55.783114  2023-07-23      LMP
4  2023-07-23T11:00:00-00:00  58.408803  2023-07-23      LMP
       INTERVALSTARTTIME_GMT        MW      OPR_DT LMP_TYPE
0  2023-07-23T07:00:00-00:00  0.201228  2023-07-23      MCC
1  2023-07-23T08:00:00-00:00  0.104237  2023-07-23      MCC
2  2023-07-23T09:00:00-00:00 -0.055910  2023-07-23      MCC
3  2023-07-23T10:00:00-00:00 -0.170398  2023-07-23      MCC
4  2023-07-23T11:00:00-00:00 -0.208846  2023-07-23      MCC
       INTERVALSTARTTIME_GMT         MW      OPR_DT LMP_TYPE
0  2023-07-23T07:00:00-00:00  68.672913  2023-07-23      MCE
1  2023-07-23T08:00:00-00:00  63.908187  2023-07-23      MCE
2  2023-07-23T09:00:00-00:00  58.131855  2023-07-23      MCE
3  2023-07-23T10:00:00-00:00  